# Coursera Capstone

### Purpose

We will group capital cities from around the world based on the local services and facilities available to residents.


### Audience

The users of this report can be ortganisation looking to expand internationally and they would like to see which countries are simialr to the ones they are already operating in. 


### Methodology 

Perform  K-NearestNeighbour analysis to group similar global capital cities.

### Data

Latitude/Longitude for Capital Cities  => https://raw.githubusercontent.com/lee-chau/Coursera_Capstone/master/worldcapitals.csv

Foursquare explore API => https://api.foursquare.com/v2/venues/explore



## Import required libraries

In [1]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
#from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

#!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library




### Create Dataframes

#### Data: Top50 Australian Cities with Geocoding

In [2]:
capital_cities_file = "https://raw.githubusercontent.com/lee-chau/Coursera_Capstone/master/worldcapitals.csv"
capital_cities_df = pd.read_csv(capital_cities_file, header=0, quotechar = '"')
print (capital_cities_df.shape)
capital_cities_df.columns = ['City','Country','lat','lng']
capital_cities_df.head()

(47, 4)


,City,Country,lat,lng
0,Amsterdam,NL,52.37403,4.88969
1,Ankara,TR,39.91987,32.85427
2,Athens,GR,37.97945,23.71622
3,Bangkok,TH,13.75000,100.51667
4,Beijing,CN,39.90750,116.39723


### Define function "getNearbyVenues" to return nearby venues from Foursquare APIs

In [3]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    # Set Constants
    CLIENT_ID = 'M0QORA2HQB1K2LGHPUZL05GIZEQZUPSY2PTWEWQGF3GR5PZS' 
    CLIENT_SECRET = 'C12HAR2WRB3QY0EZVK1EKHC4XLHY2TQLFEROQOUZHUIN3RB2' 
    VERSION = '20180605' 
    LIMIT = 100 # limit of number of venues returned by Foursquare API
    radius = 1500 # define radius

   # Section can be one of food, drinks, coffee, shops, arts, outdoors, sights, trending, nextVenues OR topPicks   
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}&section=trending'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['City', 
                  'City Latitude', 
                  'City Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [4]:
capital_city_venues = getNearbyVenues(names=capital_cities_df['City'],
                                   latitudes=capital_cities_df['lat'],
                                   longitudes=capital_cities_df['lng']
                                  )

Amsterdam
Ankara
Athens
Bangkok
Beijing
Beirut
Belgrade
Berlin
Brussels
Budapest
Buenos Aires
Cairo
Canberra
Colombo
Copenhagen
Doha
Dublin
Havana
Helsinki
Hong Kong
Jakarta
Kuala Lumpur
Lisbon
London
Luxembourg
Macau
Madrid
Mexico City
Monaco
Moscow
Nicosia
Oslo
Ottawa
Paris
Roma
Sarajevo
Seoul
Singapore
Stockholm
Tokyo
Vatican City
Victoria
Vienna
Warsaw
Washington, D. C.
Wellington
Zagreb


In [5]:
capital_city_venues.head(10)

,City,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Amsterdam,52.37403,4.88969,De Biertuin Prinsengracht,52.364023,4.885399,Beer Garden
1,Amsterdam,52.37403,4.88969,Latei,52.373292,4.900279,Café
2,Amsterdam,52.37403,4.88969,Bistrot Neuf,52.379359,4.893727,Bistro
3,Amsterdam,52.37403,4.88969,Prinsengracht,52.363179,4.887371,Canal
4,Amsterdam,52.37403,4.88969,Whisky Café L&B,52.362837,4.885202,Whisky Bar
5,Amsterdam,52.37403,4.88969,Salmuera,52.373261,4.879472,Latin American Restaurant
6,Amsterdam,52.37403,4.88969,Raw Materials - The home store,52.372335,4.876656,Furniture / Home Store
7,Amsterdam,52.37403,4.88969,Fuoco Vivo,52.371664,4.873550,Italian Restaurant
8,Amsterdam,52.37403,4.88969,Brouwersgracht,52.380767,4.887875,Canal
9,Amsterdam,52.37403,4.88969,Belgisch Bierproeflokaal De Zotte,52.366195,4.880800,Bar


In [6]:
#How many values have we had?
capital_city_venues.groupby('City').count()

,City Latitude,City Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
City,,,,,,
Amsterdam,100,100,100,100,100,100
Ankara,100,100,100,100,100,100
Athens,100,100,100,100,100,100
Bangkok,100,100,100,100,100,100
Beijing,100,100,100,100,100,100
Beirut,100,100,100,100,100,100
Belgrade,100,100,100,100,100,100
Berlin,100,100,100,100,100,100
Brussels,100,100,100,100,100,100


In [7]:
print('There are {} uniques categories.'.format(len(capital_city_venues['Venue Category'].unique())))

There are 367 uniques categories.


### One Hot Encoding

In [8]:
# one hot encoding
capital_city_onehot = pd.get_dummies(capital_city_venues[['Venue Category']], prefix="", prefix_sep="")

capital_city_onehot['City'] = capital_city_venues['City']
cols = ['City']  + [col for col in capital_city_onehot if col != 'City']
capital_city_onehot = capital_city_onehot[cols]

print(capital_city_onehot.shape)
capital_city_onehot.head()

(4349, 368)


,City,ATM,Abruzzo Restaurant,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Betawinese Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Cafeteria,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Donburi Restaurant,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kokoreç Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Macanese Restaurant,Magirio,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Padangnese Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Radio St

In [9]:
capital_city_grouped = capital_city_onehot.groupby('City').mean().reset_index()
capital_city_grouped.head()

,City,ATM,Abruzzo Restaurant,Accessories Store,Adult Boutique,African Restaurant,American Restaurant,Antique Shop,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Austrian Restaurant,Auto Garage,BBQ Joint,Bagel Shop,Bakery,Balinese Restaurant,Bar,Baseball Field,Baseball Stadium,Basketball Court,Basketball Stadium,Bath House,Beach,Beach Bar,Beer Bar,Beer Garden,Beer Store,Beijing Restaurant,Belgian Restaurant,Betawinese Restaurant,Big Box Store,Bike Rental / Bike Share,Bike Shop,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Brasserie,Bratwurst Joint,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridal Shop,Bridge,Bubble Tea Shop,Buffet,Building,Burger Joint,Burrito Place,Cafeteria,Café,Camera Store,Canal,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Casino,Castle,Cemetery,Chaat Place,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Cigkofte Place,Circus,City Hall,Climbing Gym,Clothing Store,Club House,Cocktail Bar,Coffee Shop,College Academic Building,College Cafeteria,College Gym,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Cosmetics Shop,Courthouse,Coworking Space,Creperie,Cricket Ground,Cuban Restaurant,Cultural Center,Cupcake Shop,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Dim Sum Restaurant,Diner,Discount Store,Distribution Center,Dive Bar,Donburi Restaurant,Doner Restaurant,Donut Shop,Dry Cleaner,Dumpling Restaurant,Eastern European Restaurant,Egyptian Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Event Space,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Filipino Restaurant,Fish & Chips Shop,Fish Market,Fish Taverna,Flea Market,Flower Shop,Food,Food & Drink Shop,Food Court,Food Truck,Fountain,Fraternity House,French Restaurant,Fried Chicken Joint,Friterie,Frozen Yogurt Shop,Furniture / Home Store,Gaming Cafe,Garden,Garden Center,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,German Restaurant,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gukbap Restaurant,Gym,Gym / Fitness Center,Gym Pool,Halal Restaurant,Harbor / Marina,Hawaiian Restaurant,Health & Beauty Service,Health Food Store,Hill,Historic Site,History Museum,Hobby Shop,Hong Kong Restaurant,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Hotel Pool,Hotpot Restaurant,Hungarian Restaurant,IT Services,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Juice Bar,Kafenio,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Kokoreç Restaurant,Korean Restaurant,Lake,Latin American Restaurant,Lebanese Restaurant,Library,Lighthouse,Lingerie Store,Liquor Store,Lounge,Macanese Restaurant,Magirio,Malay Restaurant,Marijuana Dispensary,Market,Martial Arts Dojo,Massage Studio,Mediterranean Restaurant,Memorial Site,Men's Store,Mexican Restaurant,Meyhane,Meze Restaurant,Middle Eastern Restaurant,Mini Golf,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Mosque,Motorcycle Shop,Mountain,Movie Theater,Multiplex,Museum,Music School,Music Store,Music Venue,Nail Salon,Neighborhood,New American Restaurant,Newsstand,Night Market,Nightclub,Nightlife Spot,Noodle House,Office,Opera House,Optical Shop,Organic Grocery,Other Great Outdoors,Other Nightlife,Outdoor Sculpture,Outdoors & Recreation,Padangnese Restaurant,Pakistani Restaurant,Palace,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Peking Duck Restaurant,Performing Arts Venue,Persian Restaurant,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pie Shop,Pier,Pizza Place,Playground,Plaza,Poke Place,Polish Restaurant,Pool,Pool Hall,Portuguese Restaurant,Pub,Public Art,Racetrack,Radio St

In [10]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [11]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['City']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
capital_city_venues_sorted = pd.DataFrame(columns=columns)
capital_city_venues_sorted['City'] = capital_city_grouped['City']

for ind in np.arange(capital_city_grouped.shape[0]):
    capital_city_venues_sorted.iloc[ind, 1:] = return_most_common_venues(capital_city_grouped.iloc[ind, :], num_top_venues)

capital_city_venues_sorted.head()

,City,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,Bar,Café,Restaurant,Coffee Shop,Marijuana Dispensary,Italian Restaurant,Cocktail Bar,Sandwich Place,Cheese Shop,Bistro
1,Ankara,Café,Dance Studio,Art Gallery,Bookstore,Coffee Shop,Music Venue,Pizza Place,Sandwich Place,Bar,Kebab Restaurant
2,Athens,Bar,Café,Greek Restaurant,Historic Site,Dessert Shop,Hotel Bar,Coffee Shop,Cocktail Bar,Nightclub,Souvlaki Shop
3,Bangkok,Noodle House,Chinese Restaurant,Dessert Shop,Asian Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Burger Joint
4,Beijing,Historic Site,Coffee Shop,Peking Duck Restaurant,History Museum,Chinese Restaurant,Fast Food Restaurant,Hotel Bar,Shopping Mall,Café,Clothing Store


### Run a K-means cluster

In [12]:
kclusters = 5 # set number of clusters

capital_city_grouped_clustering = capital_city_grouped.drop('City', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(capital_city_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([3, 3, 3, 3, 3, 3, 3, 4, 3, 3])

Cluster the top 7 venues for each capital city

In [13]:
# add clustering label
capital_city_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

capital_city_merged = capital_cities_df

capital_city_merged = capital_city_merged.join(capital_city_venues_sorted.set_index('City'), on='City')


capital_city_merged.head() 

,City,Country,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,NL,52.37403,4.88969,3,Bar,Café,Restaurant,Coffee Shop,Marijuana Dispensary,Italian Restaurant,Cocktail Bar,Sandwich Place,Cheese Shop,Bistro
1,Ankara,TR,39.91987,32.85427,3,Café,Dance Studio,Art Gallery,Bookstore,Coffee Shop,Music Venue,Pizza Place,Sandwich Place,Bar,Kebab Restaurant
2,Athens,GR,37.97945,23.71622,3,Bar,Café,Greek Restaurant,Historic Site,Dessert Shop,Hotel Bar,Coffee Shop,Cocktail Bar,Nightclub,Souvlaki Shop
3,Bangkok,TH,13.75000,100.51667,3,Noodle House,Chinese Restaurant,Dessert Shop,Asian Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Burger Joint
4,Beijing,CN,39.90750,116.39723,3,Historic Site,Coffee Shop,Peking Duck Restaurant,History Museum,Chinese Restaurant,Fast Food Restaurant,Hotel Bar,Shopping Mall,Café,Clothing Store


### Visualise the clusters

In [14]:

address = 'Europe'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Europe are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Europe are 51.0, 10.0.


In [15]:
print(capital_city_merged.shape)
capital_city_merged = capital_city_merged.dropna(axis=0, subset=['Cluster Labels'])
capital_city_merged['Cluster Labels'] = capital_city_merged['Cluster Labels'].astype(int)
print(capital_city_merged.shape)
capital_city_merged.head()

(47, 15)
(47, 15)


,City,Country,lat,lng,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,NL,52.37403,4.88969,3,Bar,Café,Restaurant,Coffee Shop,Marijuana Dispensary,Italian Restaurant,Cocktail Bar,Sandwich Place,Cheese Shop,Bistro
1,Ankara,TR,39.91987,32.85427,3,Café,Dance Studio,Art Gallery,Bookstore,Coffee Shop,Music Venue,Pizza Place,Sandwich Place,Bar,Kebab Restaurant
2,Athens,GR,37.97945,23.71622,3,Bar,Café,Greek Restaurant,Historic Site,Dessert Shop,Hotel Bar,Coffee Shop,Cocktail Bar,Nightclub,Souvlaki Shop
3,Bangkok,TH,13.75000,100.51667,3,Noodle House,Chinese Restaurant,Dessert Shop,Asian Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Burger Joint
4,Beijing,CN,39.90750,116.39723,3,Historic Site,Coffee Shop,Peking Duck Restaurant,History Museum,Chinese Restaurant,Fast Food Restaurant,Hotel Bar,Shopping Mall,Café,Clothing Store


In [16]:

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=2)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(capital_city_merged['lat'], capital_city_merged['lng'], capital_city_merged['City'], capital_city_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

### Examine the clusters

Cluster 1

In [17]:
cluster = capital_city_merged.loc[capital_city_merged['Cluster Labels'] == 0, capital_city_merged.columns[[0] + list(range(4, capital_city_merged.shape[1]))]]
print (cluster.shape)
cluster.head(cluster.shape[0])

(1, 12)


,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
17,Havana,0,Cuban Restaurant,Monument / Landmark,Hotel Bar,Middle Eastern Restaurant,Castle,Baseball Stadium,Seafood Restaurant,Breakfast Spot,Art Gallery,Caribbean Restaurant


### Cluster 2

In [18]:
cluster = capital_city_merged.loc[capital_city_merged['Cluster Labels'] == 1, capital_city_merged.columns[[0] + list(range(4, capital_city_merged.shape[1]))]]
print (cluster.shape)
cluster.head(cluster.shape[0])

(8, 12)


,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
11,Cairo,1,Café,Theater,Plaza,Falafel Restaurant,Pastry Shop,Movie Theater,Egyptian Restaurant,Swiss Restaurant,Bookstore,Clothing Store
12,Canberra,1,Café,Coffee Shop,Wine Bar,Plaza,Bar,Bookstore,Asian Restaurant,Italian Restaurant,Korean Restaurant,Shopping Mall
21,Kuala Lumpur,1,Café,Malay Restaurant,Chinese Restaurant,Indian Restaurant,Asian Restaurant,Food Truck,Convenience Store,Theater,Restaurant,Bubble Tea Shop
25,Macau,1,Café,Chinese Restaurant,Coffee Shop,Portuguese Restaurant,Cantonese Restaurant,History Museum,Dessert Shop,Hotel,Park,Macanese Restaurant
35,Sarajevo,1,Café,Fast Food Restaurant,Basketball Stadium,Electronics Store,Office,Bakery,Shopping Mall,Lake,Market,Gas Station
41,Victoria,1,Market,African Restaurant,Ice Cream Shop,Pizza Place,Farmers Market,Bar,Fast Food Restaurant,Café,Historic Site,Shopping Mall
43,Warsaw,1,Café,Cocktail Bar,Bistro,Vegetarian / Vegan Restaurant,Park,Polish Restaurant,Coffee Shop,Theater,Bar,Ice Cream Shop
46,Zagreb,1,Café,Restaurant,Bar,Plaza,Mediterranean Restaurant,Wine Bar,Historic Site,Bistro,Park,Nightclub


### Cluster 3

In [19]:
cluster = capital_city_merged.loc[capital_city_merged['Cluster Labels'] == 2, capital_city_merged.columns[[0] + list(range(4, capital_city_merged.shape[1]))]]
print (cluster.shape)
cluster.head(cluster.shape[0])

(1, 12)


,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
39,Tokyo,2,Convenience Store,Ramen Restaurant,Japanese Restaurant,Indian Restaurant,Sake Bar,Italian Restaurant,Public Art,Fast Food Restaurant,Chinese Restaurant,Café


### Cluster 4

In [20]:
cluster = capital_city_merged.loc[capital_city_merged['Cluster Labels'] == 3, capital_city_merged.columns[[0] + list(range(4, capital_city_merged.shape[1]))]]
print (cluster.shape)
cluster.head(cluster.shape[0])

(29, 12)


,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Amsterdam,3,Bar,Café,Restaurant,Coffee Shop,Marijuana Dispensary,Italian Restaurant,Cocktail Bar,Sandwich Place,Cheese Shop,Bistro
1,Ankara,3,Café,Dance Studio,Art Gallery,Bookstore,Coffee Shop,Music Venue,Pizza Place,Sandwich Place,Bar,Kebab Restaurant
2,Athens,3,Bar,Café,Greek Restaurant,Historic Site,Dessert Shop,Hotel Bar,Coffee Shop,Cocktail Bar,Nightclub,Souvlaki Shop
3,Bangkok,3,Noodle House,Chinese Restaurant,Dessert Shop,Asian Restaurant,Coffee Shop,Thai Restaurant,Seafood Restaurant,Café,Japanese Restaurant,Burger Joint
4,Beijing,3,Historic Site,Coffee Shop,Peking Duck Restaurant,History Museum,Chinese Restaurant,Fast Food Restaurant,Hotel Bar,Shopping Mall,Café,Clothing Store
5,Beirut,3,Lebanese Restaurant,Café,Coffee Shop,Mediterranean Restaurant,French Restaurant,Restaurant,Diner,Middle Eastern Restaurant,Italian Restaurant,Pharmacy
6,Belgrade,3,Restaurant,Bar,Coffee Shop,Café,Park,Ice Cream Shop,Bakery,Art Gallery,Pizza Place,Pub
8,Brussels,3,Bar,Chocolate Shop,Concert Hall,Beer Bar,Bookstore,Plaza,Coffee Shop,Bakery,Art Museum,Brasserie
9,Budapest,3,Dessert Shop,Coffee Shop,Plaza,Hungarian Restaurant,Bistro,Restaurant,Cocktail Bar,Bakery,Café,Bar
10,Buenos Aires,3,Argentinian Restaurant,Coffee Shop,Theater,Bar,Café,Italian Restaurant,Deli / Bodega,Burger Joint,Pizza Place,BBQ Joint


### Cluster 5

In [21]:
cluster = capital_city_merged.loc[capital_city_merged['Cluster Labels'] == 4, capital_city_merged.columns[[0] + list(range(4, capital_city_merged.shape[1]))]]
print (cluster.shape)
cluster.head(cluster.shape[0])

(8, 12)


,City,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
7,Berlin,4,Italian Restaurant,Coffee Shop,Bookstore,Vietnamese Restaurant,Japanese Restaurant,Restaurant,Vegetarian / Vegan Restaurant,Sandwich Place,Plaza,Bar
23,London,4,Theater,Art Gallery,Indian Restaurant,Plaza,Bookstore,Boutique,Bakery,Ice Cream Shop,Dessert Shop,English Restaurant
24,Luxembourg,4,Italian Restaurant,French Restaurant,Park,Bar,Indian Restaurant,Plaza,Pub,Bakery,German Restaurant,Trattoria/Osteria
28,Monaco,4,Italian Restaurant,French Restaurant,Restaurant,Bar,Garden,Grocery Store,Boutique,Spa,Nightclub,Plaza
33,Paris,4,French Restaurant,Plaza,Cocktail Bar,Ice Cream Shop,Restaurant,Bookstore,Pastry Shop,Creperie,Café,Historic Site
34,Roma,4,Italian Restaurant,Plaza,Ice Cream Shop,Historic Site,Sandwich Place,Pizza Place,Bookstore,Fountain,Bar,Scenic Lookout
40,Vatican City,4,Italian Restaurant,Ice Cream Shop,Sandwich Place,Pizza Place,Restaurant,Plaza,Seafood Restaurant,Museum,Café,Bakery
44,"Washington, D. C.",4,Monument / Landmark,Pizza Place,Food Truck,Art Museum,Coffee Shop,Plaza,Park,Mediterranean Restaurant,Southern / Soul Food Restaurant,Hotel Bar
